# All necessary imports

In [ ]:
import sys
sys.path.append('..')

In [ ]:
from source.code.utils import load_obj

In [ ]:
from source.code.utils import generate_pipeline
from source.code.utils import generate_cat_feature_counts
from source.code.utils import generate_features_names
from source.code.ItemSelector import ItemSelector

In [ ]:
from imblearn.pipeline import Pipeline
from sklearn.pipeline import FeatureUnion

In [ ]:
import os
import numpy as np
import pandas as pd
from tqdm import tqdm
import pandas_profiling

In [ ]:
pd.set_option('display.max_colwidth', -1)
pd.set_option('display.max_rows', 30000)
pd.set_option('display.max_columns', 30000)

In [ ]:
data_path = '../data/dataset/processed/{}.csv'
transformed_data_path = '../data/dataset/transformed/{}.csv'
meta_path = '../data/dataset/processed/meta-info/{}.pkl'

# Data reading

Firstly we just load all data into memory, then profile each dataset,

then try to filter features that are most interesting for us at the moment

(continuous, categorical, binary features without na, features with low na percentage etc.).

In [ ]:
dataset_names = ['application_train', 'application_test', 'bureau', 'bureau_balance', 'credit_card_balance', 'installments_payments', 'POS_CASH_balance', 'previous_application', 'sample_submission']

In [ ]:
data_dict = dict(zip(dataset_names, list(map(lambda name: pd.read_csv(filepath_or_buffer=data_path.format(name)), tqdm(dataset_names)))))

## Columns needed

In [ ]:
datasets_num_features = load_obj(meta_path.format('datasets_num_features'))

In [ ]:
datasets_cat_features = load_obj(meta_path.format('datasets_cat_features'))

In [ ]:
datasets_bin_features = load_obj(meta_path.format('datasets_bin_features'))

In [ ]:
extended_features_list = list(map(lambda name: generate_features_names(
    datasets_bin_features[name].tolist(),
    generate_cat_feature_counts(data_dict[name], datasets_cat_features[name].tolist()),
    datasets_num_features[name].tolist()
), tqdm(dataset_names)))

In [ ]:
pipelines = list(
    map(
        lambda name: Pipeline([
            ('union', FeatureUnion(
                [('bin', Pipeline([('choose', ItemSelector(datasets_bin_features[name].tolist()))]))] +\
                list(map(generate_pipeline, datasets_cat_features[name].tolist())) +\
                [('num', Pipeline([('choose', ItemSelector(datasets_num_features[name].tolist()))]))]
            ))]),
        tqdm(dataset_names)
    )
)

In [ ]:
datasets_tr = list(map(lambda name_number: pd.DataFrame(pipelines[name_number].fit_transform(data_dict[dataset_names[name_number]]), columns=extended_features_list[name_number]), tqdm(range(len(dataset_names)))))

In [ ]:
tr_data_dict = dict(zip(dataset_names, datasets_tr))

In [ ]:
_ = list(map(lambda name: tr_data_dict[name].to_csv(transformed_data_path.format(name), index=False), tqdm(dataset_names)))